# Assignment 1 EDA

Exploratory data analysis for the readmit30 dataset.

In [ ]:
# Environment setup for local execution
import os
import sys

base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

os.environ['TRAIN_PATH'] = os.path.join(base_dir, 'scripts', 'data', 'public', 'train.csv')
os.environ['DEV_PATH']   = os.path.join(base_dir, 'scripts', 'data', 'public', 'dev.csv')
os.environ['TEST_PATH']  = os.path.join(base_dir, 'scripts', 'data', 'public', 'public_test.csv')
os.environ['OUT_PATH']   = 'predictions.csv'



In [ ]:
# Load data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

np.random.seed(42)

TRAIN_PATH = os.environ.get("TRAIN_PATH", "../scripts/data/public/train.csv")
DEV_PATH   = os.environ.get("DEV_PATH",   "../scripts/data/public/dev.csv")
TEST_PATH  = os.environ.get("TEST_PATH",  "../scripts/data/public/public_test.csv")
OUT_PATH   = os.environ.get("OUT_PATH",   "predictions.csv")

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

assert "row_id" in train.columns and "readmit30" in train.columns
assert "row_id" in test.columns

X_train = train.drop(columns=["readmit30"])
y_train = train["readmit30"].astype(int)

---
## 1. Data Snapshot

In [ ]:
df = train.copy()
target_col = "readmit30"

# Treat common placeholder missing values as NaN
obj_cols = df.select_dtypes(include=["str"]).columns
df[obj_cols] = df[obj_cols].replace("?", np.nan)

# Basic dataset snapshot
n_rows, n_cols = df.shape
feature_dtypes = df.drop(columns=[target_col]).dtypes
num_count = feature_dtypes.apply(lambda t: pd.api.types.is_numeric_dtype(t)).sum()
cat_count = len(feature_dtypes) - num_count
readmit_rate = df[target_col].mean()

print(f"Rows x Columns: {n_rows:,} x {n_cols}")
print(f"Outcome column: {target_col}")
print(f"Readmission rate: {readmit_rate:.2%}")
print(f"Numeric features: {num_count} | Categorical features: {cat_count}")
print(f"\nTarget distribution:")
print(df[target_col].value_counts())
display(df.head())

---
## 2. Missingness Audit

In [ ]:
# Calculate missingness for all columns
missing_tbl = (
    df.isna().sum()
    .to_frame("missing_count")
    .assign(missing_pct=lambda x: x["missing_count"] / len(df) * 100)
    .sort_values("missing_pct", ascending=False)
)

display(Markdown("### Missingness Table (Top 20 by %)"))
display(missing_tbl.head(20))

# top 15 columns by % missing
top15 = missing_tbl.head(15).iloc[::-1]
plt.figure(figsize=(10, 7))
plt.barh(top15.index, top15["missing_pct"], color="#4C78A8")
plt.title("Top 15 Columns by % Missing", fontsize=14, fontweight='bold')
plt.xlabel("% Missing")
plt.tight_layout()
plt.show()

# look at columns by missingness level
acceptable_cols = missing_tbl[missing_tbl["missing_pct"] < 5].head(10).index.tolist()
problem_cols = missing_tbl[missing_tbl["missing_pct"] > 30].head(5).index.tolist()

print("\nColumns with low missingness (<5%):")
for c in acceptable_cols:
    pct = missing_tbl.loc[c, "missing_pct"]
    print(f"  {c}: {pct:.2f}%")

print("\nColumns with high missingness (>30%):")
for c in problem_cols:
    pct = missing_tbl.loc[c, "missing_pct"]
    print(f"  {c}: {pct:.2f}%")

---
## 3. Missingness vs Outcome

In [ ]:
display(Markdown("### Missingness vs Outcome Analysis"))

# Select 3 columns with interesting missingness patterns
missing_candidates = ['weight', 'A1Cresult', 'payer_code']

# Ensure columns exist
missing_candidates = [c for c in missing_candidates if c in df.columns]

missing_assoc_notes = []
readmit_table_rows = []

for col in missing_candidates:
    is_missing = df[col].isna()
    rates = df.groupby(is_missing)[target_col].mean().reindex([False, True])
    rates.index = ["Not Missing", "Missing"]
    table = rates.to_frame("readmit_rate")
    
    display(Markdown(f"#### **{col}** (Missing: {missing_tbl.loc[col, 'missing_pct']:.1f}%)"))
    display(table)
    
    # plot
    plt.figure(figsize=(5, 3.5))
    plt.bar(table.index, table["readmit_rate"], color=["#72B7B2", "#F58518"])
    plt.title(f"Readmit Rate by Missingness: {col}")
    plt.ylabel("Readmit Rate")
    plt.ylim(0, max(table["readmit_rate"]) * 1.2)
    plt.tight_layout()
    plt.show()
    
    # calculate difference
    rate_not_missing = table.loc["Not Missing", "readmit_rate"]
    rate_missing = table.loc["Missing", "readmit_rate"]
    diff = abs(rate_missing - rate_not_missing)
    
    print(f"\nReadmit rate when missing: {rate_missing:.2%} vs not missing: {rate_not_missing:.2%}")
    if diff >= 0.02:
        print(f"Missingness appears associated with outcome (diff = {diff:.2%}). Consider MAR/MNAR.\n")
    else:
        print(f"Missingness not strongly associated (diff = {diff:.2%}). Likely MCAR.\n")
    
    readmit_table_rows.append({
        "variable": col,
        "not_missing_rate_%": rate_not_missing * 100,
        "missing_rate_%": rate_missing * 100,
        "difference_%": diff * 100
    })

# Summary table (TH's format)
# summary table
readmit_comparison = pd.DataFrame(readmit_table_rows).round(2)
display(readmit_comparison)

---
## 4. Data Quality Checks

In [ ]:
display(Markdown("### Duplicate Checks"))

# check duplicates
dup_rows = df.duplicated(keep='first').sum()
print(f"Duplicate rows (excluding first occurrence): {dup_rows:,}")

# Check for repeated encounter IDs
if 'encounter_id' in df.columns:
    dup_encounters = df.duplicated(subset=['encounter_id'], keep='first').sum()
    print(f"Duplicate encounter_ids: {dup_encounters:,}")

# Check for repeated patient numbers
if 'patient_nbr' in df.columns:
    dup_patients = df.duplicated(subset=['patient_nbr'], keep='first').sum()
    print(f"Duplicate patient_nbr's: {dup_patients:,}")
    print(f"\nTop 10 repeated patient_nbr's:")
    print(df['patient_nbr'].value_counts().head(10))
    print(f"\nInterpretation: {dup_patients:,} patients have multiple encounters in the training set.")
    print("Risk: Model may see test-set data if same patients appear in train/test splits.")
    print("Recommendation: Verify train/test split is done at patient level, not encounter level.")

display(Markdown("### Numeric Outlier Analysis"))

# Select 3 key numeric columns for outlier analysis
outlier_cols = ["time_in_hospital", "num_lab_procedures", "num_medications"]
outlier_cols = [c for c in outlier_cols if c in df.columns]

if len(outlier_cols) >= 3:
    # percentile-based summary
    summary_rows = []
    for col in outlier_cols:
        p1, p99 = df[col].quantile([0.01, 0.99])
        summary_rows.append({
            "column": col,
            "min": df[col].min(),
            "p01": p1,
            "median": df[col].median(),
            "p99": p99,
            "max": df[col].max()
        })
    summary_df = pd.DataFrame(summary_rows)
    display(Markdown("#### Percentile Summary (1st/50th/99th)"))
    display(summary_df)
    
    # histograms
    fig, axes = plt.subplots(1, len(outlier_cols), figsize=(15, 4))
    for ax, col in zip(axes, outlier_cols):
        data = df[col].dropna()
        ax.hist(data, bins=30, color="#4C78A8", edgecolor="white", alpha=0.9)
        ax.set_title(f"{col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
    fig.suptitle("Distributions: Numeric Features", y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # boxplots
    fig, axes = plt.subplots(1, len(outlier_cols), figsize=(12, 3))
    for ax, col in zip(axes, outlier_cols):
        sns.boxplot(x=df[col], ax=ax, color="#A1C9F4")
        ax.set_title(col)
    fig.suptitle("Boxplots: Outlier Detection", y=1.02, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("   - Check for extreme outliers (values beyond 99th percentile)")
    print("   - Consider winsorizing or capping at 1st/99th percentiles before scaling")
    print("   - Validate that extreme values are clinically plausible (not data entry errors)")
else:
    print("Not enough numeric columns for comprehensive outlier analysis.")

---
## 5. Leakage Screening

In [ ]:
display(Markdown("### Leakage Screening"))

# keyword-based screening
leak_keywords = ["readmit", "discharge", "death", "mort", "post", "after", "length", "los", "stay"]
leakage_cols = [c for c in df.columns if c != target_col and any(k in c.lower() for k in leak_keywords)]

# Add known leakage columns from team insights
additional_suspects = ['payer_code', 'discharge_disposition_id']
for col in additional_suspects:
    if col in df.columns and col not in leakage_cols:
        leakage_cols.append(col)

def leakage_reason(col: str) -> str:
    """automated reasoning function"""
    lc = col.lower()
    if "readmit" in lc:
        return "CRITICAL: Directly references the outcome variable"
    if "discharge" in lc and "disposition" in lc:
        return "HIGH RISK: Discharge disposition known only AFTER admission ends; may indicate deceased/transferred"
    if "discharge" in lc:
        return "HIGH RISK: May include post-discharge information"
    if "death" in lc or "mort" in lc:
        return "CRITICAL: Post-outcome indicator (deceased patients cannot be readmitted)"
    if "payer" in lc or "insurance" in lc:
        return "MODERATE RISK: Insurance/payer info finalized AFTER discharge; temporal leakage possible"
    if "length" in lc or "los" in lc or "stay" in lc:
        return "MODERATE RISK: Length of stay determined DURING/AFTER admission; may correlate with discharge timing"
    if "post" in lc or "after" in lc:
        return "HIGH RISK: Post-event timing indicator"
    return "REVIEW: Could encode post-encounter information"

print(f"Found {len(leakage_cols)} potential leakage columns:\n")
for col in leakage_cols[:10]:  # Show first 10
    reason = leakage_reason(col)
    print(f"  - {col}")
    print(f"    {reason}\n")

---
## 6. Final Summary

- Top 5 missing columns: weight (96.9%), max_glu_serum (94.7%), A1Cresult (83.4%), medical_specialty (48.9%), payer_code (39.6%)
- Overall readmission rate: 10.87%
- A1Cresult missingness associated with outcome (11.45% missing vs 9.73% not missing) - encode as 'not_measured' indicator
- Weight not strongly associated with outcome (diff < 1%) but 97% missing - drop it
- Payer_code weak association (11.53% vs 10.92%) but temporal leakage concern - drop it
- No duplicate rows found, but multiple encounters per patient - verify patient-level split
- Outliers in time_in_hospital, num_lab_procedures, num_medications - consider capping at 1st/99th percentiles
- discharge_disposition_id is post-discharge info (home/SNF/expired) - drop to avoid leakage

**Next steps:**
- Drop: weight, max_glu_serum (too sparse), discharge_disposition_id and payer_code (leakage risk)
- Keep A1Cresult but encode missingness as binary indicator
- Impute remaining missing with median (numeric) or mode (categorical)